#### 패키지 Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

#### 전역변수 설정

In [2]:
trainGraphTitle = "Train Data"
testGraphTitle = "Test Data"
checkpointPath = "./weightCheckpoint/bestParam.hdf5"
loss = "mse" 

resultComment = "Optimizer - SGD"
fileSavePath = "./result/Optimizer/SGD"

depth = "SingleGRU" # Default
# depth = "DoubleGRU" 
# depth = "TripleGRU"

# 변수 "" 가 Default
hiddenState = 32 # units: 16 "32" 64
timeStep = 20 # input_length 10 "20" 40
activation = "tanh" # "tanh" sigmoid
epochs = 100 # 50 "100" 200
batchSize = 64 # 32 "64" 256
dataSize = 10 # 5 "10" 40
optimizer = "sgd" # "adam" sgd
patience = 30 # 10 "30" "50"

parameters = f"""
layer = {depth}
hiddenState = {hiddenState}
timeStep = {timeStep}
activation = {activation}
epochs = {epochs}
batchSize = {batchSize}
dataSetYear = {dataSize}
optimizer = {optimizer}
patience = {patience}
"""

#### Pandas Setting

In [3]:
pd.set_option('display.max_rows', None) # row 생략 없이 출력
pd.set_option('display.max_columns', None) # col 생략 없이 출력

#### Data Load

In [4]:
apple = pd.read_csv("Apple_5Y.csv")

if dataSize == 10:
    apple = pd.read_csv("Apple_10Y.csv")
elif dataSize == 40:
    apple = pd.read_csv("Apple_Whole_Period.csv")

#### Describe 확인

In [5]:
apple.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2.518000e+03
mean,62.383387,63.087693,61.696664,62.416599,60.637030,1.713540e+08
std,48.683035,49.322128,48.051043,48.709794,49.248659,1.250498e+08
min,13.856071,14.271429,13.753571,13.947500,12.064585,3.519590e+07
25%,26.971249,27.219376,26.705000,26.983750,24.676634,9.188698e+07
50%,41.353750,41.872500,40.888750,41.313750,39.511671,1.301368e+08
75%,91.272499,92.881876,90.824377,91.526875,90.122925,2.045116e+08
max,182.630005,182.940002,179.119995,182.009995,180.959732,1.460852e+09


#### trainData, testData 가공하는 함수

In [6]:
def transformData(data: [[float]]):
    # 날짜 제외
    data = data.drop(columns=["Date"])

    # 데이터 분리 test: 200개와 나머지
    trainSet = data[ : -200]
    testSet = data[-200 : ]
    
    # 데이터 0~1로 정규화
    sc = MinMaxScaler(feature_range=(0, 1)) 
    sc.fit(trainSet)
    trainSet = sc.transform(trainSet)
    testSet = sc.transform(testSet)
    
    # trainX, trainY, testX, testY 분리
    def parsingData(dataSet: [[float]]) -> ([[[float]]], [[float]]):
        dataX, dataY = [], []
        for index in range(len(dataSet) - timeStep):
            temp = []
            for step in range(timeStep):
                temp.append(dataSet[index + step])
            dataX.append(temp)
            dataY.append(dataSet[index + timeStep])

        return np.array(dataX), np.array(dataY)

    trainDataX, trainDataY = parsingData(trainSet)
    testDataX, testDataY = parsingData(testSet)
    
    
    return trainDataX, trainDataY, testDataX[:-30], testDataY, sc

#### Data Parsing

In [7]:
trainX, trainY, testX, testY, scaler = transformData(apple)

#### Model Implementation

In [8]:
model = Sequential()

if depth == "TripleGRU":
    model.add(
        GRU(
            units = hiddenState,
            input_length = trainX.shape[1],
            input_dim = trainX.shape[2],
            activation = activation,
            return_sequences = True
        )
    )
    model.add(
        GRU(
            units = hiddenState,
            activation = activation,
            return_sequences = True
        )
    )
    model.add(
        GRU(
            units = hiddenState,
            activation = activation
        )
    )
    
    model.add(Dense(6))
    
elif depth == "DoubleGRU":
    model.add(
        GRU(
            units = hiddenState,
            input_length = trainX.shape[1],
            input_dim = trainX.shape[2],
            activation = activation,
            return_sequences = True
        )
    )
    model.add(
        GRU(
            units = hiddenState,
            activation = activation
        )
    )
    
    model.add(Dense(6))
elif depth == "SingleGRU":
    model.add(
    GRU(
        units = hiddenState,
        input_length = trainX.shape[1],
        input_dim = trainX.shape[2],
        activation = activation
        )
    )
    model.add(Dense(6))

model.summary()

Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 32)                3840      
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 4,038
Trainable params: 4,038
Non-trainable params: 0
_________________________________________________________________


2023-01-05 11:55:57.088760: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-05 11:55:57.088861: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


#### Model Complie

In [9]:
model.compile(
    loss = loss,
    optimizer = optimizer,
    metrics = ["mae"]
)

#### Model Training

In [10]:
earlyStop = EarlyStopping(
    monitor = 'loss',
    min_delta = 0.0001,
    patience = patience,
    verbose = 1
)
saveBest = ModelCheckpoint(
    filepath = checkpointPath,
    monitor = "loss",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,
    mode = "auto",
    save_freq = "epoch"
)

fitStartTime = time.time()
history = model.fit(
    trainX,
    trainY,
    epochs = epochs,
    batch_size = batchSize,
    callbacks=[earlyStop, saveBest]
)
fitEndTime = time.time()


model.load_weights(checkpointPath)

Epoch 1/100


2023-01-05 11:55:57.297839: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-05 11:55:57.896388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-05 11:55:57.985685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11/36 [========>.....................] - ETA: 0s - loss: 0.1138 - mae: 0.2283

2023-01-05 11:55:58.104666: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


36/36 [==============================] - ETA: 0s - loss: 0.0847 - mae: 0.1877
Epoch 1: loss improved from inf to 0.08466, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 1s 13ms/step - loss: 0.0847 - mae: 0.1877
Epoch 2/100
34/36 [===========================>..] - ETA: 0s - loss: 0.0474 - mae: 0.1382
Epoch 2: loss improved from 0.08466 to 0.04761, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 0s 10ms/step - loss: 0.0476 - mae: 0.1390
Epoch 3/100
33/36 [==========================>...] - ETA: 0s - loss: 0.0328 - mae: 0.1219
Epoch 3: loss improved from 0.04761 to 0.03239, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 0s 10ms/step - loss: 0.0324 - mae: 0.1217
Epoch 4/100
36/36 [==============================] - ETA: 0s - loss: 0.0248 - mae: 0.1137
Epoch 4: loss improved from 0.03239 to 0.02484, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [========

32/36 [=========================>....] - ETA: 0s - loss: 0.0012 - mae: 0.0199
Epoch 31: loss improved from 0.00122 to 0.00120, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 0s 11ms/step - loss: 0.0012 - mae: 0.0199
Epoch 32/100
36/36 [==============================] - ETA: 0s - loss: 0.0012 - mae: 0.0196
Epoch 32: loss improved from 0.00120 to 0.00118, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 0s 10ms/step - loss: 0.0012 - mae: 0.0196
Epoch 33/100
34/36 [===========================>..] - ETA: 0s - loss: 0.0012 - mae: 0.0194
Epoch 33: loss improved from 0.00118 to 0.00116, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 0s 10ms/step - loss: 0.0012 - mae: 0.0194
Epoch 34/100
33/36 [==========================>...] - ETA: 0s - loss: 0.0011 - mae: 0.0191
Epoch 34: loss improved from 0.00116 to 0.00115, saving model to ./weightCheckpoint/bestParam.hdf5
36/3

35/36 [============================>.] - ETA: 0s - loss: 0.0010 - mae: 0.0176
Epoch 61: loss improved from 0.00105 to 0.00104, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 0s 10ms/step - loss: 0.0010 - mae: 0.0175
Epoch 62/100
33/36 [==========================>...] - ETA: 0s - loss: 0.0010 - mae: 0.0175 
Epoch 62: loss improved from 0.00104 to 0.00104, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 0s 10ms/step - loss: 0.0010 - mae: 0.0175
Epoch 63/100
35/36 [============================>.] - ETA: 0s - loss: 0.0010 - mae: 0.0175    
Epoch 63: loss improved from 0.00104 to 0.00104, saving model to ./weightCheckpoint/bestParam.hdf5
36/36 [==============================] - 0s 10ms/step - loss: 0.0010 - mae: 0.0175
Epoch 64/100
34/36 [===========================>..] - ETA: 0s - loss: 0.0010 - mae: 0.0175
Epoch 64: loss improved from 0.00104 to 0.00104, saving model to ./weightCheckpoint/bestParam.hdf5

#### Test Data + 30일 후 예측

In [11]:
thirtyPredict = testX

for _ in range(30):
    currentPredict = model.predict(thirtyPredict)
    recentPredict = np.reshape(currentPredict[-1], (1, currentPredict[-1].shape[0]))
    newPredictStep = thirtyPredict[-1, 1:]
    nextPredict = np.append(newPredictStep, recentPredict, axis = 0)
    nextPredict = np.reshape(nextPredict, (1, nextPredict.shape[0], nextPredict.shape[1]))
    thirtyPredict = np.append(thirtyPredict, nextPredict, axis = 0)

1/5 [=====>........................] - ETA: 0s

2023-01-05 11:56:24.454212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-05 11:56:24.482592: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 11ms/step


#### 시간 및 평가 기록

In [22]:
fitTime = fitEndTime - fitStartTime
score = model.evaluate(testX, testY[:-30])

print(testX.shape)
print(testY.shape)
print(thirtyPredict.shape)

print(thirtyPredict[-30:].shape)
print(testY[-30:].shape)

predictScore = model.evaluate(thirtyPredict[-30:0], testY[-30:])


f = open("result.txt", "a")
f.write(f"{resultComment}\n모델 학습 시간: {fitTime:.3} sec\n평가 손실: {score[0]}\n30일 예측 손실: {predictScore[0]}\n{parameters}\n--------------------------------------\n\n")
f.close()

5/5 [==============================] - 0s 8ms/step - loss: 8.1450e-04 - mae: 0.0223
(150, 20, 6)
(180, 6)
(180, 20, 6)
(30, 20, 6)
(30, 6)


ValueError: Data cardinality is ambiguous:
  x sizes: 0
  y sizes: 30
Make sure all arrays contain the same number of samples.

In [14]:
f"모델 학습 시간: {fitTime:.3} sec"

'모델 학습 시간: 27.1 sec'

In [ ]:
f"평가 손실: {score[0]}"

#### 예측

In [ ]:
trainPrediction = scaler.inverse_transform(model.predict(trainX))
testPrediction = scaler.inverse_transform(model.predict(testX))

inversingTrainY = scaler.inverse_transform(trainY)
inversingTestY = scaler.inverse_transform(testY)

### 그래프

#### Loss

In [ ]:
loss = history.history["loss"]
plt.title("Loss")
plt.plot(loss, label="loss")
plt.grid(True)
plt.savefig(f"{fileSavePath}/Loss.png")
plt.legend()
plt.show()

#### Train Data Graph

In [ ]:
plt.title(trainGraphTitle)
plt.plot(inversingTrainY[:, 4], label="Train ADJ.Close")
plt.plot(trainPrediction[:, 4], label="Predict ADJ.Close")
plt.grid(True)
plt.savefig(f"{fileSavePath}/Train.png")
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()

#### Test Data Graph

In [ ]:
# plt.title(testGraphTitle)
# plt.plot(inversingTestY[:, 4], label="Test ADJ.Close")
# plt.plot(testPrediction[:, 4], label="Predict ADJ.Close")
# plt.grid(True)
# plt.xlabel('Day')
# plt.ylabel('Price')
# plt.axvline(x=len(testY)-30, color='green', linestyle='-', linewidth=1)
# plt.legend()
# plt.savefig(f"{fileSavePath}/Test.png")
# plt.show()

#### Test Data + 30일 예측 그래프

In [ ]:
thirtyDaysAfterpredict = scaler.inverse_transform(model.predict(thirtyPredict))

plt.title("30 Days After")
plt.plot(inversingTestY[:, 4], label="Test ADJ.Close")
plt.plot(thirtyDaysAfterpredict[:, 4], label="30 Days predict ADJ.Close")
plt.grid(True)
plt.xlabel('Day')
plt.ylabel('Price')
plt.axvline(x=len(testY)-30, color='green', linestyle='-', linewidth=1)
plt.legend()
plt.savefig(f"{fileSavePath}/30Predict.png")
plt.show()